# Programa Nº 1:

-----

## WordCloud Exploration

1. WordCloud I<br />
    1.1. Preprocesamiento y Procesamiento<br />
        - Preprocesamiento: Feature extracción, ajuste del content en su conjunto, tokenización<br />
        - Procesamiento: Implementación de filtros para la limpieza y filtros para la selección de muestras<br />
    1.2. Tratamiento y Generación de WordCloud<br />
        - Tratamiento: Aplicación de stop_words por medio de modelo IA (spacy)<br />
        - Generación WC: Uno para la feature "content_wc" y otro para la feature "token_wc"<br />
    1.3 Agregación:<br />
        - Agregación en el sentido de la estructura de datos, el programa recibe un fichero con extensión<br />
            ".csv" y devuelve una lista Python, el primer elemento es "dataframes" y el segundo "wordcloud_storage"<br />
        - dataframes: Es un diccionario Python, en su interior hay 1 pandas.DataFrame por cada filtro booleano<br />
            se haya implementado. Por default no hay filtros booleanos implementados, por lo tanto el diccionario<br />
            tendría la siguiente forma ```{'csv_filename': [pd.DataFrame(...)]}``` con todo el contenido del batch.<br />
        - wordcloud_storage: Es un diccionario Python, en su interior posee las instancias de clase del objeto tipo<br />
            WordCloud de la librería wordcloud. La App utiliza dos features, "content_wc", y "token_wc", por lo tanto,<br />
            siguiendo con el ejemplo de no aplicar ningún filtro booleano, el segundo elemento de la lista de agregación<br />
            sería así ```{'csv_filename': [ [WC_content, WC_token] ] }```.<br />
        - El output del set de aplicaciones resulta entonces, ```result = [{...}, {...}]``` - el primer diccionario es<br />
            dataframes, y el segundo wordcloud_storage<br />
    1.4 Outputs:<br />
        - El programa fue pensado para obtener un primer approach a lo que el contenido refleja en su conjunto<br />
        - Optimizar la extracción de muestras por medio de multi-hilos, durante el proceso de stop_words<br />
        - wordcloud.WordCloud permite obtener imágenes, texto, y campos vectoriales por medio de la implementación<br />
            de sus métodos internos, to_image(), to_svg(), to_array(), respectivamente.<br />
        - Brindar al usuario flexibilidad en los outputs, por medio de implementación de máscaras, filtros de palabras,<br />
            configuraciónes preseteadas, y demás.<br />
        - Un resplado en .txt de la última configuración utilizada para cada máscara.<br />

2. Clasificación de Sentimiento y Emociones<br />
    2.1 Prepocesamiento y Procesamiento<br />
        - Preprocesamiento: Feature extracción, ajuste del content en su conjunto, tokenización<br />
        - Procesamiento: Time Series adjustment, agregación de variables temporales<br />
    2.2 Transformers:<br />
        - Multiprocesos: Paralelización los modelos de Emociones<br />
        - Sentimiento: 3 modelos M1_I, M1_II, y M1_III<br />
            + M1_I: Optimización por multihilos, sub-batches secuenciales<br />
            + M1_II: Ingesta por Data Streaming mediante Python Generators<br />
            + M1_III: Ingesta por Data Streaming mediante Python Generators<br />
        - Emociones: 2 modelos M2, y M3<br />
            + M2: Ingesta por Data Streaming mediante Python Generators<br />
            + M3: Ingesta por Data Streaming mediante Python Generators<br />
    2.3 Analíticas:<br />
        - Gráficos: Línea de tiempo, Gráficos Radiales, Gráficos de Barras<br />
        - Outputs: Ficheros .csv y .txt con resumen descriptivo<br />

3. WordCloud II:<br />
    3.1 Lectura de clasificaciones<br />
        - Definir filtros booleanos para cada muestra<br />
    3.2 Tratamiento y Generación de WordCloud<br />
        - Tratamiento: Extracción con filtros, implementación de multi-hilos para stop_words<br />
        - Generación WC: Una lista por cada muestra y un objeto por cada feature.<br />
    3.3 Agregación:<br />
    3.4 Outputs<br />


4. Identificación de Tópicos: es otra notebook que creo que se llama Lotto.
            M4: Optimización por multihilos, sub-batches paralelos


In [4]:
import pickle

with open("___main___results.pkl", "rb") as f:
    result = pickle.load(f)

In [10]:
len(result)

2